In [ ]:
import numpy as np
from random import shuffle
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

## Data sets

In [ ]:
def f(x: np.array) -> np.array:
    """
        DEFINE YOUR PREFERRED FUNCTIONS HERE 
    """
    return np.cos(10*x)

num_examples = 5000
validation_fraction = 0.1
test_fraction = 0.1

validation_set_size = int(num_examples*validation_fraction)
test_set_size = int(num_examples*test_fraction)
training_set_size = num_examples - validation_set_size - test_set_size

assert type(num_examples)==int and num_examples>0
assert training_set_size>0

all_x = np.random.uniform(0*np.pi, np.pi, (1, num_examples)).T
np.random.shuffle(all_x)
x_training, x_validation, x_test = (all_x[:training_set_size], 
                                    all_x[training_set_size: training_set_size + validation_set_size],
                                    all_x[training_set_size + validation_set_size:])
(y_training, y_validation, y_test) = (f(x) for x in (x_training, x_validation, x_test))

## NN parameters

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()  
        self.fc1 = nn.Linear(1, 50)  
        self.fc2 = nn.Linear(50, 50)
        self.fc3 = nn.Linear(50, 50)
        self.fc4 = nn.Linear(50, 50)
        self.fc5 = nn.Linear(50, 1)
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.relu(self.fc4(x))
        x = self.fc5(x)
        return x
net = Net()

In [ ]:
params = list(net.parameters())
print(len(params))

In [ ]:
input = torch.tensor(x_training, dtype=torch.float, requires_grad=True)
target = torch.tensor(y_training, dtype=torch.float)
criterion = nn.MSELoss()
training_epoch = 1000
epoch_digit = len(list(str(training_epoch)))
display_step = int(training_epoch * 0.1)
display_precision = 6
learning_rate = 0.001
weight_decay = 1/4000
optimizer = optim.Adam(net.parameters(), lr=learning_rate, weight_decay=weight_decay)

## Train the NN

In [ ]:
for epoch in range(1, training_epoch+1): 
    optimizer.zero_grad()
    net.zero_grad()
    outputs = net(input)
    loss = criterion(outputs, target)
    loss.backward()
    optimizer.step()

    running_loss = loss.item()
    if not epoch % display_step:    # print every display step
        print('[%0*d] loss: %.*f' %
                (epoch_digit, epoch, display_precision, running_loss))
print('Finished Training')

## Test the NN

In [ ]:
with torch.no_grad():
    out = net(input)
    fig = plt.figure(figsize=(8,5))
    plt.scatter(input.detach().numpy(), out.detach().numpy(), c='r', label='prediction on the training set', s=5)
    plt.scatter(input.detach().numpy(), y_training, c='b', label='labels', s=5)
    plt.legend()
    plt.show()

In [ ]:
inputs = torch.tensor(x_test, dtype=torch.float)
with torch.no_grad():
    out = net(inputs)
    fig = plt.figure(figsize=(8,5))
    plt.scatter(inputs.detach().numpy(), out.detach().numpy(), c='r', label='prediction on the test set', s=5)
    plt.legend()
    plt.show()